<a href="https://colab.research.google.com/github/datasistah/ml_sytem_design_course/blob/main/airline_tweet_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Twitter Sentiment Analysis

**Problem statement:** Airline industry had a very hard time post covid to sustain their business due to a long hault. It is very important for them to make sure they exceed customer expectations. The best way to evaluate performance is customer feedback. You are given a dataset of airline tweets from real customers.

A sentiment analysis job about the problems of each major U.S. airline. Twitter data was scraped from February of 2015 and contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as "late flight" or "rude service").

You will use the text column and sentiment column to create a classification model that classifies a given tweet into one of the 3 classes - positive, negative, neutral.

**Understanding the Dataset:**

Dataset contains many columns out of which below are most important ones-
1. airline_sentiment - defines the sentiment of the tweet 
2. negative_reason - reason for the negative feedback (if negative)
3. Text - tweet text content
4. tweet_location - location from which tweet was posted

You can use more columns in your model training if you want. 


**Steps to perform**
1. Load dataset - https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment
2. Clean, preprocess data and EDA
3. Vectorise columns that contain text 
4. Run Classification model to classify - positive, negative or neutral
5. Evaluate model



## Steps to Download kaggle datasets using Kaggle Public API

1. Go to your account, Scroll to API section and Click Expire API Token to remove previous tokens

2. Click on Create New API Token - It will download kaggle.json file on your machine.

In [1]:
#Making a change to track

In [2]:
#!pip install -q kaggle

^C
ERROR: Operation cancelled by user


In [3]:
# from google.colab import files
# files.upload()

In [4]:
# ! mkdir ~/.kaggle

# ! cp kaggle.json ~/.kaggle/

In [5]:
#! chmod 600 ~/.kaggle/kaggle.json

In [6]:
#! kaggle competitions download -c 'name-of-competition'

In [8]:
#pandas profile https://pypi.org/project/pandas-profiling/
import pandas as pd
from ydata_profiling import ProfileReport

/Users/teasletx/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [10]:
pwd

'/Users/teasletx/Desktop/Learning/ML_System_Design/ml_sytem_design_course'

### Load dataset 

In [30]:
file = 'kaggle/Tweets.csv'
df = pd.read_csv(file)
df.head()


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [31]:
print(f"The length of the original dataset is {len(df)}")

The length of the original dataset is 14640


In [32]:
#profile = ProfileReport(df, title="EDA: Pandas Profiling Report")
#profile.to_file("airline_tweet_sentiment_EDA.html")

In [33]:
#profile

In [34]:
#important columns
imp_cols = ['tweet_id','airline_sentiment','negativereason','text','tweet_location']
df = df[imp_cols]
df.head()

,tweet_id,airline_sentiment,negativereason,text,tweet_location
0,570306133677760513,neutral,NaN,@VirginAmerica What @dhepburn said.,NaN
1,570301130888122368,positive,NaN,@VirginAmerica plus you've added commercials t...,NaN
2,570301083672813571,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...,Lets Play
3,570301031407624196,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...,NaN
4,570300817074462722,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...,NaN


In [35]:
df = df.drop_duplicates()
df.head()

,tweet_id,airline_sentiment,negativereason,text,tweet_location
0,570306133677760513,neutral,NaN,@VirginAmerica What @dhepburn said.,NaN
1,570301130888122368,positive,NaN,@VirginAmerica plus you've added commercials t...,NaN
2,570301083672813571,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...,Lets Play
3,570301031407624196,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...,NaN
4,570300817074462722,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...,NaN


In [29]:
print(f"The length after droping duplicates {len(df)}")

The length after droping duplicates 14604


In [36]:
df.airline_sentiment.value_counts()

negative    9118
neutral     3074
positive    2340
Name: airline_sentiment, dtype: int64

#### Data Cleaning

In [42]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def clean_tweet(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove user mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize and remove stop words
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Join the tokens back into a single string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text



[nltk_data] Downloading package punkt to /Users/teasletx/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [43]:
df['clean_tweet'] = df['text'].apply(lambda x: clean_tweet(x))
df.head()

,tweet_id,airline_sentiment,negativereason,text,tweet_location,clean_tweet
0,570306133677760513,neutral,NaN,@VirginAmerica What @dhepburn said.,NaN,said
1,570301130888122368,positive,NaN,@VirginAmerica plus you've added commercials t...,NaN,plus youve added commercials experience tacky
2,570301083672813571,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...,Lets Play,didnt today must mean need take another trip
3,570301031407624196,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...,NaN,really aggressive blast obnoxious entertainmen...
4,570300817074462722,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...,NaN,really big bad thing


In [45]:
#create labels for sentiment
possible_labels = df.airline_sentiment.unique()

label_dict = {}
for index, label in enumerate(possible_labels):
    label_dict[label] = index
label_dict

{'neutral': 0, 'positive': 1, 'negative': 2}

In [47]:
#create column for sentiment label as int
df['sentiment_label'] = df['airline_sentiment'].apply(lambda x: label_dict[x])
df.head()

,tweet_id,airline_sentiment,negativereason,text,tweet_location,clean_tweet,sentiment_label
0,570306133677760513,neutral,NaN,@VirginAmerica What @dhepburn said.,NaN,said,0
1,570301130888122368,positive,NaN,@VirginAmerica plus you've added commercials t...,NaN,plus youve added commercials experience tacky,1
2,570301083672813571,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...,Lets Play,didnt today must mean need take another trip,0
3,570301031407624196,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...,NaN,really aggressive blast obnoxious entertainmen...,2
4,570300817074462722,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...,NaN,really big bad thing,2


In [48]:
df.airline_sentiment.value_counts()

negative    9118
neutral     3074
positive    2340
Name: airline_sentiment, dtype: int64

In [50]:
#use stratified sampling for imbalanced dataset, there are more negative tweets than positive or neutral

# import necessary libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [52]:
tweets = list(df.clean_tweet)
tweets[:2]

['said', 'plus youve added commercials experience tacky']

In [53]:
# create a count vectorizer to convert the text into numerical features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(tweets)

In [59]:
sentiments = list(df.sentiment_label)
sentiments[:2]

[0, 1]

In [60]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, sentiments, test_size=0.2, random_state=42, stratify = df['sentiment_label'].values)



In [72]:
import numpy as np
# convert the target arrays to NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)


In [73]:
print(f" X_train.shape = {X_train.shape}\n X_test.shape = {X_test.shape}\n y_train = {y_train.shape}\n y_test = {y_test.shape}")

 X_train.shape = (11625, 13024)
 X_test.shape = (2907, 13024)
 y_train = (11625,)
 y_test = (2907,)


In [78]:
from sklearn.linear_model import LogisticRegression
# create a logistic regression model with increased max_iter
model = LogisticRegression(max_iter=1000)


In [79]:
#fit model to training data
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [80]:
# make predictions on the testing data
y_pred = model.predict(X_test)


In [89]:
import pickle
# save the model to disk
filename = 'twitter_classification_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [90]:
# evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7791537667698658


In [93]:
# define a single tweet to predict the sentiment of
tweet = "I hate going to work on Mondays"

# convert the tweet to numerical features using the same vectorizer
X_tweet = vectorizer.transform([tweet])

# make a prediction using the trained model
sentiment = model.predict(X_tweet)

In [94]:
print(sentiment)

[2]


In [82]:
from sklearn.metrics import classification_report

# print precision, recall, and F1 score for each class
print(classification_report(y_test, y_pred, target_names=possible_labels))


              precision    recall  f1-score   support

     neutral       0.61      0.56      0.58       615
    positive       0.76      0.65      0.70       468
    negative       0.83      0.89      0.86      1824

    accuracy                           0.78      2907
   macro avg       0.73      0.70      0.71      2907
weighted avg       0.77      0.78      0.77      2907



In [83]:
### Use SpaCy to train model

In [95]:
#!pip install spacy

In [98]:
#!python -m spacy download en_core_web_sm


In [102]:
#pip install transformers -U 

In [2]:
import spacy

/Users/teasletx/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
#!pip install tokenizers==0.9.4

In [5]:
#!pip install transformers -U


In [6]:
#!pip install -U transformers

In [3]:
nlp = spacy.load('en_core_web_sm')

In [9]:
import pandas as pd

In [10]:
file = 'kaggle/Tweets.csv'
df = pd.read_csv(file)
df.head()


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [11]:
print(f"The length of the original dataset is {len(df)}")

The length of the original dataset is 14640


In [12]:
#important columns
imp_cols = ['tweet_id','airline_sentiment','negativereason','text','tweet_location']
df = df[imp_cols]
df.head()

,tweet_id,airline_sentiment,negativereason,text,tweet_location
0,570306133677760513,neutral,NaN,@VirginAmerica What @dhepburn said.,NaN
1,570301130888122368,positive,NaN,@VirginAmerica plus you've added commercials t...,NaN
2,570301083672813571,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...,Lets Play
3,570301031407624196,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...,NaN
4,570300817074462722,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...,NaN


In [13]:
df = df.drop_duplicates()
df.head()

,tweet_id,airline_sentiment,negativereason,text,tweet_location
0,570306133677760513,neutral,NaN,@VirginAmerica What @dhepburn said.,NaN
1,570301130888122368,positive,NaN,@VirginAmerica plus you've added commercials t...,NaN
2,570301083672813571,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...,Lets Play
3,570301031407624196,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...,NaN
4,570300817074462722,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...,NaN


In [14]:
print(f"The length after droping duplicates {len(df)}")

The length after droping duplicates 14532


In [15]:
df.airline_sentiment.value_counts()

negative    9118
neutral     3074
positive    2340
Name: airline_sentiment, dtype: int64

#### Data Cleaning

In [16]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def clean_tweet(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove user mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize and remove stop words
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Join the tokens back into a single string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text



[nltk_data] Downloading package punkt to /Users/teasletx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
df['clean_tweet'] = df['text'].apply(lambda x: clean_tweet(x))
df.head()

,tweet_id,airline_sentiment,negativereason,text,tweet_location,clean_tweet
0,570306133677760513,neutral,NaN,@VirginAmerica What @dhepburn said.,NaN,said
1,570301130888122368,positive,NaN,@VirginAmerica plus you've added commercials t...,NaN,plus youve added commercials experience tacky
2,570301083672813571,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...,Lets Play,didnt today must mean need take another trip
3,570301031407624196,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...,NaN,really aggressive blast obnoxious entertainmen...
4,570300817074462722,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...,NaN,really big bad thing


In [18]:
df.airline_sentiment.value_counts()

negative    9118
neutral     3074
positive    2340
Name: airline_sentiment, dtype: int64

In [19]:
#Create tuples to train using SpaCy
spacy_data = []
for index, row in df.iterrows():
    tweet = row['text']
    sentiment = row['airline_sentiment']
    data = (tweet,sentiment)
    spacy_data.append(data)
spacy_data[:2]

[('@VirginAmerica What @dhepburn said.', 'neutral'),
 ("@VirginAmerica plus you've added commercials to the experience... tacky.",
  'positive')]

In [20]:
df['spacy_data'] = spacy_data
df.head()

,tweet_id,airline_sentiment,negativereason,text,tweet_location,clean_tweet,spacy_data
0,570306133677760513,neutral,NaN,@VirginAmerica What @dhepburn said.,NaN,said,"(@VirginAmerica What @dhepburn said., neutral)"
1,570301130888122368,positive,NaN,@VirginAmerica plus you've added commercials t...,NaN,plus youve added commercials experience tacky,(@VirginAmerica plus you've added commercials ...
2,570301083672813571,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...,Lets Play,didnt today must mean need take another trip,(@VirginAmerica I didn't today... Must mean I ...
3,570301031407624196,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...,NaN,really aggressive blast obnoxious entertainmen...,(@VirginAmerica it's really aggressive to blas...
4,570300817074462722,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...,NaN,really big bad thing,(@VirginAmerica and it's a really big bad thin...


In [48]:
type(clean_tweet)

function

In [50]:
#one hot encoding for sentiment
y = pd.get_dummies(df['airline_sentiment'], prefix='label')

In [51]:
y

,label_negative,label_neutral,label_positive
0,0,1,0
1,0,0,1
2,0,1,0
3,1,0,0
4,1,0,0
...,...,...,...
14635,0,0,1
14636,1,0,0
14637,0,1,0
14638,1,0,0


In [55]:
#create labels for sentiment
possible_labels = df.airline_sentiment.unique()

label_dict = {}
for index, label in enumerate(possible_labels):
    label_dict[label] = index
label_dict

{'neutral': 0, 'positive': 1, 'negative': 2}

In [56]:
df.head()

,tweet_id,airline_sentiment,negativereason,text,tweet_location,clean_tweet,spacy_data
0,570306133677760513,neutral,NaN,@VirginAmerica What @dhepburn said.,NaN,said,"(@VirginAmerica What @dhepburn said., neutral)"
1,570301130888122368,positive,NaN,@VirginAmerica plus you've added commercials t...,NaN,plus youve added commercials experience tacky,(@VirginAmerica plus you've added commercials ...
2,570301083672813571,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...,Lets Play,didnt today must mean need take another trip,(@VirginAmerica I didn't today... Must mean I ...
3,570301031407624196,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...,NaN,really aggressive blast obnoxious entertainmen...,(@VirginAmerica it's really aggressive to blas...
4,570300817074462722,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...,NaN,really big bad thing,(@VirginAmerica and it's a really big bad thin...


In [57]:
df['label'] = df.airline_sentiment.apply(lambda x: label_dict[x])
df.head()

,tweet_id,airline_sentiment,negativereason,text,tweet_location,clean_tweet,spacy_data,label
0,570306133677760513,neutral,NaN,@VirginAmerica What @dhepburn said.,NaN,said,"(@VirginAmerica What @dhepburn said., neutral)",0
1,570301130888122368,positive,NaN,@VirginAmerica plus you've added commercials t...,NaN,plus youve added commercials experience tacky,(@VirginAmerica plus you've added commercials ...,1
2,570301083672813571,neutral,NaN,@VirginAmerica I didn't today... Must mean I n...,Lets Play,didnt today must mean need take another trip,(@VirginAmerica I didn't today... Must mean I ...,0
3,570301031407624196,negative,Bad Flight,@VirginAmerica it's really aggressive to blast...,NaN,really aggressive blast obnoxious entertainmen...,(@VirginAmerica it's really aggressive to blas...,2
4,570300817074462722,negative,Can't Tell,@VirginAmerica and it's a really big bad thing...,NaN,really big bad thing,(@VirginAmerica and it's a really big bad thin...,2


In [60]:
data = tuple(zip(df['clean_tweet'].tolist(), df['airline_sentiment'].tolist()))
data[1]

('plus youve added commercials experience tacky', 'positive')

In [1]:
type(data[1])

NameError: name 'data' is not defined

In [66]:
import tqdm
nlp = spacy.load('en_core_web_sm')
docs=[]
for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
    if(label=='positive'):
        doc.cats['positive']=1
        doc.cats['negative']=0
        doc.cats['neutral'] =0
    elif(label=='negative'):
        doc.cats['positive']=0
        doc.cats['negative']=1
        doc.cats['neutral'] =0
    else:
        doc.cats['positive']=0
        doc.cats['negative']=0
        doc.cats['neutral'] =1
    docs.append(doc)
docs

TypeError: 'module' object is not callable